In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Define the directory containing the dataset
dataset_dir = '/content/drive/MyDrive/Hackathons/HackForCreators/Recommendation_Systems/Art_RS/munch_paintings'

In [4]:
# Define the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

In [5]:
# Add a Global Average Pooling layer to the model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [6]:
# Add a Dense layer with 1024 units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)

In [7]:
# Add a Dropout layer to prevent overfitting
x = tf.keras.layers.Dropout(0.5)(x)

In [8]:
# Add a final Dense layer with the number of classes in the dataset
num_classes = len(os.listdir(dataset_dir))
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

In [9]:
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
# Define the image size for resizing images
img_size = (224, 224)

In [12]:
# Define a function to preprocess images
def preprocess_image(image_path):
    # Load the image and resize it
    img = load_img(image_path, target_size=img_size)
    # Convert the image to a NumPy array
    x = img_to_array(img)
    # Expand the dimensions of the image from (img_size, img_size, 3) to (1, img_size, img_size, 3)
    x = np.expand_dims(x, axis=0)
    # Preprocess the image using the preprocess_input function of the ResNet50 model
    x = preprocess_input(x)
    return x

In [13]:
# Define a function to get the top k most similar images for a given image
def get_similar_images(image_path, k=5):
    # Preprocess the query image
    query_image = preprocess_image(image_path)
    # Get the features of the query image using the pre-trained ResNet50 model
    query_features = model.predict(query_image)
    # Get the features of all the images in the dataset
    image_features = []
    for image_name in os.listdir(dataset_dir):
        image_path = os.path.join(dataset_dir, image_name)
        image = preprocess_image(image_path)
        image_feature = model.predict(image)
        image_features.append(image_feature)
    # Compute the similarity scores between the query image and all the images in the dataset
    similarity_scores = []
    for image_feature in image_features:
        similarity_score = np.dot(query_features, image_feature.T)[0][0]
        similarity_scores.append(similarity_score)
    # Get the indices of the top k most similar images
    indices = np.argsort(similarity_scores)[::-1][:k]
    # Get the paths of the top k most similar images
    similar_images = [os.path.join(dataset_dir, os.listdir(dataset_dir)[i]) for i in indices]
    return similar_images

In [14]:
img = "/content/6.jpg"

In [ ]:
recommend_art = get_similar_images(img)
print(recommend_art)

1/1 [==============================] - ETA: 0s